In [1]:
import pandas as pd
import pickle
import os
import re
import scipy.stats as st
import numpy as np
from matplotlib import pyplot as plt  
import statsmodels.api as sm
import statsmodels.formula.api as smf



In [2]:
df = pd.read_csv('adult19.csv')
df['FAMINCTC_A'].tail(30)

31967     65000
31968     25000
31969     85000
31970     10000
31971    110000
31972     52000
31973     17560
31974     12000
31975     75000
31976     45000
31977     85000
31978     15000
31979     35000
31980     18000
31981     15000
31982     40000
31983    100000
31984     36000
31985    220000
31986     65000
31987     35000
31988     61500
31989     25000
31990     29800
31991     68000
31992    116204
31993     68000
31994     60000
31995    101000
31996     79000
Name: FAMINCTC_A, dtype: int64